In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.api import VAR

# Set a random seed for reproducibility
np.random.seed(42)

# Number of observations and variables
n_obs = 2000
n_vars = 2  # You can adjust the number of variables as needed

# Intercept (mean)
intercept = np.array([0.5, -0.3])

# Create 5 lag coefficient matrices for a VAR(5)
# We'll keep them stable by making the coefficients relatively small and decreasing as lag increases
A1 = np.array([[0.3, 0.0],
               [0.0, 0.2]])
A2 = np.array([[0.05, 0.0],
               [0.0, 0.05]])
A3 = np.array([[0.02, 0.01],
               [0.0, 0.01]])
A4 = np.array([[0.01, 0.0],
               [0.0, 0.01]])
A5 = np.array([[0.01, 0.0],
               [0.0, 0.01]])

# Stack them in a list for convenience
A_matrices = [A1, A2, A3, A4, A5]

# Simulate the VAR(5) process
data = np.zeros((n_obs, n_vars))
eps = np.random.randn(n_obs, n_vars)

# Start with zeros; first 5 points won't have full lag history, so we start from t=5
for t in range(5, n_obs):
    # X_t = intercept + sum_{i=1 to 5} A_i * X_{t-i} + eps_t
    x_t = intercept.copy()
    for i, A_i in enumerate(A_matrices, start=1):
        x_t += data[t - i] @ A_i.T
    x_t += eps[t]
    data[t] = x_t

df = pd.DataFrame(data, columns=[f"var{i}" for i in range(n_vars)])

# Fit the model and select order
model = VAR(df)
# Try selecting up to lag 10 to ensure we can recover the true order of 5
order_results = model.select_order(maxlags=15)
print("Order Selection Results:")
print(order_results.summary())

# Extract the selected order based on AIC for demonstration
selected_order = order_results.aic
if selected_order is None or np.isnan(selected_order):
    selected_order = 1  # fallback if needed

print(f"Selected order based on AIC: {selected_order}")

# Fit the VAR model using the selected order
results = model.fit(selected_order)
print("\nFitted VAR Model Summary:")
print(results.summary())

# Check the estimated coefficients
print("\nEstimated Coefficients:")
print(results.coefs)


Order Selection Results:
 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0       0.1372      0.1428       1.147      0.1392
1   -0.0003659    0.01654*      0.9996   0.005844*
2   -0.003897*     0.02428     0.9961*    0.006454
3    -0.003108     0.03634      0.9969     0.01138
4    -0.002224     0.04850      0.9978     0.01641
5    0.0004570     0.06245       1.000     0.02323
6     0.001063     0.07433       1.001     0.02797
7     0.001505     0.08604       1.002     0.03256
8     0.004256      0.1001       1.004     0.03945
9     0.007004      0.1141       1.007     0.04634
10    0.008658      0.1270       1.009     0.05213
11    0.009614      0.1392       1.010     0.05723
12     0.01307      0.1540       1.013     0.06482
13     0.01297      0.1651       1.013     0.06886
14     0.01382      0.1773       1.014     0.07385
15     0.01567      0.1904       1.016     0.07984
------